# Assay Agent Workforce — Full Demo Notebook

This notebook demonstrates every agent in the Assay system, triggers each one to do real work,
and shows how the **Conductor** orchestrates the whole workforce into a daily digest.

## Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    CONDUCTOR (Orchestrator)                  │
│  Runs daily — delegates to sub-agents — compiles digest     │
└──────────┬────────┬────────┬────────┬────────┬─────────────┘
           │        │        │        │        │
    ┌──────▼──┐ ┌───▼────┐ ┌▼──────┐ ┌▼──────┐ ┌▼──────────┐
    │Engineer │ │Sentinel│ │Advocate│ │Telemetry│ │Productizer│
    │Gap Ana- │ │QA Gate-│ │GitHub │ │Usage   │ │Sales Kits │
    │lysis    │ │keeper  │ │Triage │ │Metrics │ │Generator  │
    └─────────┘ └────────┘ └───────┘ └────────┘ └───────────┘
    ┌──────────┐ ┌─────────┐ ┌─────────────┐
    │Marketing │ │Support  │ │Presentation │
    │Social    │ │FAQ Bot  │ │PowerPoint   │
    │Drafts    │ │         │ │Generator    │
    └──────────┘ └─────────┘ └─────────────┘
```

**All agents run in `dry_run=True` mode** — they report what they would do without side effects.

---
## 0. Setup — Add Project Root to Path

In [ ]:
import sys
import os
from pathlib import Path

# In Jupyter, __file__ doesn't exist. Use cwd-based detection.
# This notebook lives in notebooks/ → project root is the parent.
_notebook_dir = Path(os.path.abspath(""))
PROJECT_ROOT = _notebook_dir.parent if _notebook_dir.name == "notebooks" else _notebook_dir

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Set the GitHub repo so all agents resolve it without a redirect.
# (os.environ takes precedence over the hardcoded default in agents/config.py)
os.environ.setdefault("GITHUB_REPO", "MyDataTrends/Assay")

# Also load .env if present
try:
    from dotenv import load_dotenv
    load_dotenv(PROJECT_ROOT / ".env", override=False)
    print("✅ .env loaded")
except ImportError:
    print("ℹ️  python-dotenv not installed — .env not loaded")

# Verify LLM availability
from llm_manager.llm_interface import is_llm_available, get_active_model_name
llm_ok = is_llm_available()
print(f"{'✅' if llm_ok else '⚠️ '} LLM available: {llm_ok} — active model: {get_active_model_name()}")
print(f"📁 Project root: {PROJECT_ROOT}")
print(f"📦 GitHub repo:  {os.environ['GITHUB_REPO']}")

---
## 1. Generate Demo Data

We create a synthetic **retail e-commerce** dataset that several agents will use.
This mimics a real upload without needing external data.

In [9]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import random

# Seed for reproducibility
np.random.seed(42)
random.seed(42)

# ── Synthetic Retail Dataset ──────────────────────────────────────────────────
n = 500
categories = ["Electronics", "Clothing", "Home & Garden", "Sports", "Books"]
regions = ["North", "South", "East", "West", "Central"]
channels = ["Online", "In-Store", "Mobile App"]

dates = [datetime(2025, 1, 1) + timedelta(days=random.randint(0, 364)) for _ in range(n)]

df_retail = pd.DataFrame({
    "order_id":        range(10001, 10001 + n),
    "order_date":      [d.strftime("%Y-%m-%d") for d in dates],
    "customer_id":     np.random.randint(1000, 2000, n),
    "product_category": np.random.choice(categories, n, p=[0.30, 0.25, 0.20, 0.15, 0.10]),
    "revenue":         np.round(np.random.lognormal(4.5, 0.8, n), 2),
    "units_sold":      np.random.randint(1, 15, n),
    "discount_pct":    np.round(np.random.uniform(0, 0.35, n), 2),
    "region":          np.random.choice(regions, n),
    "channel":         np.random.choice(channels, n, p=[0.55, 0.30, 0.15]),
    "customer_rating": np.random.choice([1, 2, 3, 4, 5], n, p=[0.03, 0.07, 0.15, 0.35, 0.40]),
    "is_returned":     np.random.choice([0, 1], n, p=[0.88, 0.12]),
    "days_to_ship":    np.random.randint(1, 8, n),
})

# Save to User_Data (for Productizer)
user_data_dir = PROJECT_ROOT / "User_Data"
user_data_dir.mkdir(exist_ok=True)
demo_csv_path = user_data_dir / "demo_retail_sales.csv"
df_retail.to_csv(demo_csv_path, index=False)

print(f"✅ Created demo dataset: {demo_csv_path}")
print(f"   Shape: {df_retail.shape}")
print(f"   Columns: {list(df_retail.columns)}")
df_retail.head()

✅ Created demo dataset: c:\Projects\Assay\Assay\User_Data\demo_retail_sales.csv
   Shape: (500, 12)
   Columns: ['order_id', 'order_date', 'customer_id', 'product_category', 'revenue', 'units_sold', 'discount_pct', 'region', 'channel', 'customer_rating', 'is_returned', 'days_to_ship']


,order_id,order_date,customer_id,product_category,revenue,units_sold,discount_pct,region,channel,customer_rating,is_returned,days_to_ship
0,10001,2025-11-24,1102,Books,132.11,8,0.01,East,Online,4,0,3
1,10002,2025-02-27,1435,Electronics,319.39,11,0.05,West,In-Store,4,0,1
2,10003,2025-01-13,1860,Electronics,195.60,12,0.08,South,In-Store,3,0,6
3,10004,2025-05-21,1270,Home & Garden,160.23,1,0.34,North,Online,1,1,2
4,10005,2025-05-06,1106,Clothing,67.92,9,0.20,Central,Online,3,0,1


---
## 2. MCP Dashboard Run — Live Tool Invocation

This section creates a live **MCP session** and invokes the full Assay tool chain —
the exact same calls the Streamlit dashboard makes behind the scenes:

```
Session → describe_dataset → suggest_visualizations → create_chart × N
       → create_correlation_heatmap → run_regression → run_classification
       → detect_anomalies → forecast
```

> All cells use `await` (top-level async — supported natively by Jupyter kernels).

In [10]:
import asyncio
import json
import plotly.graph_objects as go
import plotly.express as px

from mcp_server.session import MCPSession
from mcp_server.tools.analysis import DescribeDatasetTool
from mcp_server.tools.workflow import StartWorkflowTool

print("═" * 60)
print("  MCP DASHBOARD — Session Setup & Dataset Profile")
print("═" * 60)

# ── 1. Create a local MCP session (no HTTP server needed) ─────────
mcp_session = MCPSession(user_id="notebook-demo")
mcp_session.add_dataset("retail", df_retail)
print(f"\n✅ Session created: {mcp_session.session_id[:8]}...")
print(f"   Dataset 'retail': {df_retail.shape[0]:,} rows × {df_retail.shape[1]} columns")

# ── 2. Start a workflow ────────────────────────────────────────────
wf_result = await StartWorkflowTool().execute({
    "user_id": "notebook-demo",
    "file_name": "demo_retail_sales.csv",
    "description": "Full retail analysis pipeline",
}, session=mcp_session)
wf_id = wf_result["data"]["workflow_id"]
print(f"   Workflow: {wf_id}")

# ── 3. Profile the dataset ─────────────────────────────────────────
desc_result = await DescribeDatasetTool().execute(
    {"dataset_id": "retail"}, session=mcp_session
)

if desc_result.get("success"):
    p = desc_result["data"]
    print(f"\n📊 describe_dataset →")
    for k, v in p.items():
        if k in ("sample", "plotly_json", "stats"):
            continue
        if isinstance(v, (dict, list)) and len(str(v)) > 100:
            print(f"   {k}: {{...}}")
        else:
            print(f"   {k}: {v}")
    # Show numeric summary for key revenue columns
    stats = p.get("stats", {})
    for col in ["revenue", "units_sold", "customer_rating", "discount_pct"]:
        if col in stats:
            s = stats[col]
            print(f"   [{col}] mean={s.get('mean',0):.2f}  std={s.get('std',0):.2f}"
                  f"  min={s.get('min',0):.2f}  max={s.get('max',0):.2f}")
else:
    print(f"⚠️  describe_dataset: {desc_result.get('error')}")

════════════════════════════════════════════════════════════
  MCP DASHBOARD — Session Setup & Dataset Profile
════════════════════════════════════════════════════════════

✅ Session created: a49481f5...
   Dataset 'retail': 500 rows × 12 columns
   Workflow: wf_cf22d512

📊 describe_dataset →
   shape: {'rows': 500, 'columns': 12}
   columns: {...}
   memory_mb: 0.15
   missing_total: 0


In [11]:
from mcp_server.tools.visualization import SuggestVisualizationsTool

print("═" * 60)
print("  MCP: suggest_visualizations  [Section 1 — Chart Selection]")
print("═" * 60)

suggest_result = await SuggestVisualizationsTool().execute(
    {
        "dataset_id": "retail",
        "question": "What are the key revenue drivers and return rate patterns?",
    },
    session=mcp_session,
)

if suggest_result.get("success"):
    mcp_suggestions = suggest_result["data"]["suggestions"]
    print(f"\n✅ {len(mcp_suggestions)} visualization suggestions selected:\n")
    for i, s in enumerate(mcp_suggestions, 1):
        chart_type = s.get("type", "?").upper()
        x = s.get("x", s.get("column", "—"))
        y = s.get("y", "—")
        reason = s.get("reason", "")
        print(f"  {i}. [{chart_type}]  x={x}  y={y}")
        if reason:
            print(f"     → {reason}")
else:
    print(f"⚠️  suggest_visualizations: {suggest_result.get('error')}")
    # Heuristic fallback so downstream cells still work
    mcp_suggestions = [
        {"type": "bar",       "x": "product_category", "y": "revenue",
         "reason": "Compare categories by total revenue"},
        {"type": "scatter",   "x": "discount_pct",     "y": "revenue",
         "reason": "Discount impact on revenue"},
        {"type": "histogram", "column": "customer_rating",
         "reason": "Rating distribution"},
        {"type": "pie",       "column": "channel",
         "reason": "Sales channel proportions"},
        {"type": "bar",       "x": "region",           "y": "is_returned",
         "reason": "Return rate by region"},
    ]
    print(f"\n[Fallback: {len(mcp_suggestions)} heuristic suggestions]")
    for i, s in enumerate(mcp_suggestions, 1):
        print(f"  {i}. [{s['type'].upper()}]  {s.get('reason','')}")

print(f"\n⟶  Section 2 (next cell) will render exactly these {len(mcp_suggestions)} charts via create_chart.")

════════════════════════════════════════════════════════════
  MCP: suggest_visualizations  [Section 1 — Chart Selection]
════════════════════════════════════════════════════════════


HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"



✅ 6 visualization suggestions selected:

  1. [BAR]  x=product_category  y=revenue
     → With categories like Books and Electronics visible in the data, this will show which product categories generate the most revenue. The revenue column has high variance (std: 132.8, range: 5.18-1140.88) suggesting significant differences across categories.
  2. [SCATTER]  x=discount_pct  y=revenue
     → Discount percentages range from 0% to 35% with a mean of 17.1%. This scatter plot will reveal the relationship between discounting strategy and revenue generation, helping identify optimal discount levels.
  3. [BAR]  x=region  y=is_returned
     → With 11% overall return rate (mean: 0.11) and multiple regions (East, West, South), this visualization will identify which regions have higher return rates, revealing potential quality or shipping issues by geography.
  4. [SCATTER]  x=order_id  y=revenue
     → Compare two numeric variables
  5. [HEATMAP]  x=—  y=—
     → Show correlations between nume

In [12]:
from mcp_server.tools.visualization import CreateChartTool

print("═" * 60)
print("  MCP: create_chart  [Section 2 — Render Suggested Charts]")
print("═" * 60)
print(f"\n⟶  Calling create_chart for each of the {len(mcp_suggestions)} suggestions above...\n")

create_tool = CreateChartTool()
charts_rendered = []

for i, suggestion in enumerate(mcp_suggestions):
    chart_type = suggestion.get("type", "bar")
    title = suggestion.get(
        "title",
        f"{chart_type.title()} — {suggestion.get('reason', f'Chart {i+1}')}"
    )

    # Map suggestion fields → create_chart parameters
    args = {"dataset_id": "retail", "chart_type": chart_type, "title": title}
    if "x" in suggestion:
        args["x"] = suggestion["x"]
    if "y" in suggestion:
        args["y"] = suggestion["y"]
    if "column" in suggestion:          # histogram / pie use 'column' not 'x'
        args["x"] = suggestion["column"]
    if "color" in suggestion:
        args["color"] = suggestion["color"]

    result = await create_tool.execute(args, session=mcp_session)

    if result.get("success"):
        chart_data  = result["data"]
        plotly_json = chart_data.get("plotly_json")
        chart_id    = chart_data.get("chart_id", f"chart_{i}")

        if plotly_json:
            fig = go.Figure(json.loads(plotly_json))
            fig.show()
            charts_rendered.append(chart_id)
            print(f"  ✅ {i+1}/{len(mcp_suggestions)}: [{chart_type.upper()}] {title}")
        else:
            print(f"  ⚠️  {i+1}: no plotly_json — {chart_data}")
    else:
        print(f"  ⚠️  {i+1}: create_chart failed — {result.get('error')}")

print(f"\n✅ Rendered {len(charts_rendered)} / {len(mcp_suggestions)} charts via MCP create_chart")
print(f"   Session chart store: {list(mcp_session.charts.keys())}")

════════════════════════════════════════════════════════════
  MCP: create_chart  [Section 2 — Render Suggested Charts]
════════════════════════════════════════════════════════════

⟶  Calling create_chart for each of the 6 suggestions above...



  ✅ 1/6: [BAR] Total Revenue by Product Category


  ✅ 2/6: [SCATTER] Revenue vs Discount Percentage


  ✅ 3/6: [BAR] Return Rate by Region


  ✅ 4/6: [SCATTER] Scatter — Compare two numeric variables


  ✅ 5/6: [HEATMAP] Heatmap — Show correlations between numeric columns


  ✅ 6/6: [HISTOGRAM] Histogram — Show distribution

✅ Rendered 6 / 6 charts via MCP create_chart
   Session chart store: ['chart_a8d80449', 'chart_68825425', 'chart_e67ccbea', 'chart_40d04a89', 'chart_83a7ed78', 'chart_995887b8']


In [13]:
from mcp_server.tools.visualization import CreateCorrelationHeatmapTool
from mcp_server.tools.analysis import (
    RunRegressionTool, RunClassificationTool,
    DetectAnomaliesTool, ForecastTool,
)

# ─────────────────────────────────────────────────────────────────
# create_correlation_heatmap
# ─────────────────────────────────────────────────────────────────
print("═" * 60)
print("  MCP: create_correlation_heatmap")
print("═" * 60)

hm_result = await CreateCorrelationHeatmapTool().execute(
    {"dataset_id": "retail"}, session=mcp_session
)
if hm_result.get("success"):
    hm = hm_result["data"]
    go.Figure(json.loads(hm["plotly_json"])).show()
    strong = hm.get("strong_correlations", [])
    if strong:
        print(f"\n🔗 Strong correlations (|r| > 0.5):")
        for c in strong:
            print(f"   {c['col1']} ↔ {c['col2']}: r={c['corr']:.3f}")
    else:
        print("\n  No correlations above |r| = 0.5 found.")
    print(f"\n✅ Heatmap chart_id: {hm['chart_id']}")
else:
    print(f"⚠️  create_correlation_heatmap: {hm_result.get('error')}")

# ─────────────────────────────────────────────────────────────────
# run_regression  →  predict: revenue
# ─────────────────────────────────────────────────────────────────
print("\n" + "═" * 60)
print("  MCP: run_regression  →  target: revenue")
print("═" * 60)

reg_result = await RunRegressionTool().execute(
    {"dataset_id": "retail", "target": "revenue"}, session=mcp_session
)
if reg_result.get("success"):
    r = reg_result["data"]
    print(f"\n✅ Model: {r.get('model_id','?')}")
    print(f"   R²:  {r.get('r2_score', 0):.4f}")
    print(f"   MAE: {r.get('mae', 0):.4f}")
    fi = r.get("feature_importance", {})
    if fi:
        fi_df = pd.DataFrame(list(fi.items()), columns=["feature", "importance"]).sort_values("importance")
        fig = px.bar(fi_df, x="importance", y="feature", orientation="h",
                     title="Feature Importance — Revenue Regression",
                     labels={"importance": "Importance", "feature": "Feature"})
        fig.update_layout(template="plotly_dark")
        fig.show()
else:
    print(f"⚠️  run_regression: {reg_result.get('error')}")

# ─────────────────────────────────────────────────────────────────
# run_classification  →  predict: is_returned
# ─────────────────────────────────────────────────────────────────
print("\n" + "═" * 60)
print("  MCP: run_classification  →  target: is_returned")
print("═" * 60)

cls_result = await RunClassificationTool().execute(
    {"dataset_id": "retail", "target": "is_returned"}, session=mcp_session
)
if cls_result.get("success"):
    r = cls_result["data"]
    print(f"\n✅ Model: {r.get('model_id','?')}")
    print(f"   Accuracy: {r.get('accuracy', 0):.1%}")
    fi = r.get("feature_importance", {})
    if fi:
        fi_df = pd.DataFrame(list(fi.items()), columns=["feature", "importance"]).sort_values("importance")
        fig = px.bar(fi_df, x="importance", y="feature", orientation="h",
                     title="Feature Importance — Return Prediction",
                     color="importance", color_continuous_scale="Reds",
                     labels={"importance": "Importance", "feature": "Feature"})
        fig.update_layout(template="plotly_dark")
        fig.show()
else:
    print(f"⚠️  run_classification: {cls_result.get('error')}")

# ─────────────────────────────────────────────────────────────────
# detect_anomalies  →  returns {"anomalies": {col: {count, pct}}}
# ─────────────────────────────────────────────────────────────────
print("\n" + "═" * 60)
print("  MCP: detect_anomalies")
print("═" * 60)

anom_result = await DetectAnomaliesTool().execute(
    {"dataset_id": "retail", "columns": ["revenue", "units_sold", "discount_pct"]},
    session=mcp_session,
)
if anom_result.get("success"):
    r = anom_result["data"]
    anomalies = r.get("anomalies", {})          # {col: {"count": N, "pct": P}}
    method    = r.get("method", "iqr")
    total     = sum(v["count"] for v in anomalies.values())

    print(f"\n✅ Method: {method}  |  Total anomalies across columns: {total}")
    if anomalies:
        print("   Per-column:")
        for col, info in anomalies.items():
            print(f"     {col}: {info['count']} anomalies ({info['pct']:.1f}%)")

        # Quick scatter: revenue vs units_sold, highlighting revenue outliers
        if "revenue" in anomalies:
            q1, q3 = df_retail["revenue"].quantile([0.25, 0.75])
            iqr = q3 - q1
            df_plot = df_retail.copy()
            df_plot["anomaly"] = (
                (df_retail["revenue"] < q1 - 1.5 * iqr) |
                (df_retail["revenue"] > q3 + 1.5 * iqr)
            )
            fig = px.scatter(df_plot, x="units_sold", y="revenue", color="anomaly",
                             title=f"Revenue Anomalies ({method.upper()} method)",
                             color_discrete_map={True: "#ef5350", False: "#90caf9"})
            fig.update_layout(template="plotly_dark")
            fig.show()
    else:
        print("   No anomalies detected.")
else:
    print(f"⚠️  detect_anomalies: {anom_result.get('error')}")

# ─────────────────────────────────────────────────────────────────
# forecast  →  revenue, 30 periods (moving average trend)
# ─────────────────────────────────────────────────────────────────
print("\n" + "═" * 60)
print("  MCP: forecast  →  revenue, 30 periods")
print("═" * 60)

forecast_result = await ForecastTool().execute({
    "dataset_id": "retail",
    "target":     "revenue",
    "periods":    30,
}, session=mcp_session)

if forecast_result.get("success"):
    r = forecast_result["data"]
    method = r.get("method", "?")
    vals   = r.get("forecast", [])
    print(f"\n✅ Method: {method}  |  Periods: {r.get('periods', len(vals))}")

    if vals:
        # Show historical daily average + forecast
        hist_avg = df_retail["revenue"].mean()
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=list(range(-10, 0)), y=[hist_avg] * 10,
            mode="lines", name="Historical avg", line=dict(color="#90caf9", dash="dash")
        ))
        fig.add_trace(go.Scatter(
            x=list(range(len(vals))), y=vals,
            mode="lines+markers", name="Forecast",
            line=dict(color="#42a5f5"), marker=dict(size=5)
        ))
        fig.update_layout(
            title="Revenue Forecast — next 30 periods (moving average trend)",
            xaxis_title="Period (0 = now)", yaxis_title="Revenue ($)",
            template="plotly_dark",
        )
        fig.show()
else:
    print(f"⚠️  forecast: {forecast_result.get('error')}")

# ─────────────────────────────────────────────────────────────────
# Session summary
# ─────────────────────────────────────────────────────────────────
print("\n" + "═" * 60)
print("  MCP Session Summary")
print("═" * 60)
info = mcp_session.to_dict()
print(f"\n   Session:      {info['session_id'][:8]}...")
print(f"   Datasets:     {[d['dataset_id'] for d in info['datasets']]}")
print(f"   Workflows:    {info['workflows']}")
print(f"   Charts:       {len(info['charts'])} generated")
print(f"   Models:       {len(info['models'])} trained")
print(f"   Tool calls:   {info['tool_call_count']}")

if mcp_session.charts:
    print(f"\n🎨 Charts in session:")
    for cid, chart in mcp_session.charts.items():
        meta = chart.get("metadata", {})
        print(f"   • {cid}  [{meta.get('type','?')}]  {meta.get('title', '')}")

if mcp_session.models:
    print(f"\n🤖 Models trained:")
    for mid, m in mcp_session.models.items():
        meta = m.get("metadata", {})
        print(f"   • {mid}  type={meta.get('type','?')}  target={meta.get('target','?')}")

════════════════════════════════════════════════════════════
  MCP: create_correlation_heatmap
════════════════════════════════════════════════════════════



  No correlations above |r| = 0.5 found.

✅ Heatmap chart_id: corr_0636692a

════════════════════════════════════════════════════════════
  MCP: run_regression  →  target: revenue
════════════════════════════════════════════════════════════

✅ Model: reg_f316d45d
   R²:  -0.8570
   MAE: 98.5807



════════════════════════════════════════════════════════════
  MCP: run_classification  →  target: is_returned
════════════════════════════════════════════════════════════

✅ Model: clf_d24537a5
   Accuracy: 87.0%



════════════════════════════════════════════════════════════
  MCP: detect_anomalies
════════════════════════════════════════════════════════════

✅ Method: iqr  |  Total anomalies across columns: 25
   Per-column:
     revenue: 25 anomalies (5.0%)



════════════════════════════════════════════════════════════
  MCP: forecast  →  revenue, 30 periods
════════════════════════════════════════════════════════════

✅ Method: moving_average_trend  |  Periods: 30



════════════════════════════════════════════════════════════
  MCP Session Summary
════════════════════════════════════════════════════════════

   Session:      a49481f5...
   Datasets:     ['retail']
   Workflows:    ['wf_cf22d512']
   Charts:       7 generated
   Models:       2 trained
   Tool calls:   0

🎨 Charts in session:
   • chart_a8d80449  [bar]  Total Revenue by Product Category
   • chart_68825425  [scatter]  Revenue vs Discount Percentage
   • chart_e67ccbea  [bar]  Return Rate by Region
   • chart_40d04a89  [scatter]  Scatter — Compare two numeric variables
   • chart_83a7ed78  [heatmap]  Heatmap — Show correlations between numeric columns
   • chart_995887b8  [histogram]  Histogram — Show distribution
   • corr_0636692a  [correlation]  

🤖 Models trained:
   • reg_f316d45d  type=regression  target=revenue
   • clf_d24537a5  type=classification  target=is_returned


---
## 3. Agent Setup — Load All Agents

We instantiate every agent in dry-run mode.

In [14]:
from agents.base import AgentConfig, AgentResult, Priority
from agents.conductor import ConductorAgent
from agents.engineer import EngineerAgent
from agents.sentinel import SentinelAgent
from agents.advocate import AdvocateAgent
from agents.telemetry import TelemetryAgent
from agents.productizer import ProductizerAgent
from agents.marketing import MarketingAgent
from agents.support import SupportAgent
from agents.presentation import PresentationAgent

def make_config(name, enabled=True, dry_run=True, extra=None):
    return AgentConfig(
        name=name,
        enabled=enabled,
        dry_run=dry_run,
        llm_model="claude-3-5-sonnet",
        extra=extra or {}
    )

# Instantiate all agents
agents = {
    "engineer":     EngineerAgent(config=make_config("engineer")),
    "sentinel":     SentinelAgent(config=make_config("sentinel", extra={"min_confidence_score": 7})),
    "advocate":     AdvocateAgent(config=make_config("advocate", extra={"stale_days": 14})),
    "telemetry":    TelemetryAgent(config=make_config("telemetry")),
    "productizer":  ProductizerAgent(config=make_config("productizer")),
    "marketing":    MarketingAgent(config=make_config("marketing")),
    "support":      SupportAgent(config=make_config("support")),
    "presentation": PresentationAgent(config=make_config("presentation")),
}

print("Assay Agent Workforce")
print("═" * 55)
emoji_map = {
    "engineer": "🔧", "sentinel": "🛡️", "advocate": "🤝",
    "telemetry": "📊", "productizer": "💼", "marketing": "📣",
    "support": "💬", "presentation": "📑",
}
role_map = {
    "engineer": "Codebase gap analysis & improvement recommender",
    "sentinel": "QA gatekeeper — tests, lint, confidence scoring",
    "advocate": "GitHub issue triage & community management",
    "telemetry": "Usage analytics & health monitoring",
    "productizer": "Vertical sales kit generator",
    "marketing": "Social media draft generator from git log",
    "support": "FAQ-backed intelligent issue responder",
    "presentation": "PowerPoint generator from sales kits",
}
for name, agent in agents.items():
    e = emoji_map.get(name, "•")
    mode = "[dry-run]" if agent.is_dry_run else "[LIVE]"
    print(f"  {e} {name:14s} {mode}  {role_map.get(name, '')}")
print()
print(f"  All {len(agents)} agents ready.")

Assay Agent Workforce
═══════════════════════════════════════════════════════
  🔧 engineer       [dry-run]  Codebase gap analysis & improvement recommender
  🛡️ sentinel       [dry-run]  QA gatekeeper — tests, lint, confidence scoring
  🤝 advocate       [dry-run]  GitHub issue triage & community management
  📊 telemetry      [dry-run]  Usage analytics & health monitoring
  💼 productizer    [dry-run]  Vertical sales kit generator
  📣 marketing      [dry-run]  Social media draft generator from git log
  💬 support        [dry-run]  FAQ-backed intelligent issue responder
  📑 presentation   [dry-run]  PowerPoint generator from sales kits

  All 8 agents ready.


---
## 4. Engineer Agent — Codebase Gap Analysis

The Engineer reads subsystem source files and scores their maturity against best-in-class standards.
It writes the report to `knowledge_base/product/vision_gap_analysis.md`.

In [15]:
import time

print("═" * 60)
print("  ENGINEER AGENT — Gap Analysis")
print("═" * 60)

start = time.time()
engineer_result = agents["engineer"].run()
elapsed = time.time() - start

status = "✅" if engineer_result.success else "❌"
print(f"\n{status} Status: {'Success' if engineer_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {engineer_result.summary}")

if engineer_result.metrics:
    print("\n📊 Metrics:")
    for k, v in engineer_result.metrics.items():
        if isinstance(v, float):
            print(f"   {k}: {v:.2f}")
        else:
            print(f"   {k}: {v}")

print("\n🔧 Actions taken:")
for action in engineer_result.actions_taken:
    print(f"   → {action}")

if engineer_result.escalations:
    print("\n⚠️  Escalations:")
    for esc in engineer_result.escalations:
        emoji = {"urgent": "🔴", "review": "🟡", "fyi": "🟢", "metric": "📊"}.get(esc.priority.value, "•")
        print(f"   {emoji} [{esc.priority.value.upper()}] {esc.title}")
        print(f"      {esc.detail[:200]}")

if engineer_result.error:
    print(f"\n⚠️  Error: {engineer_result.error}")

════════════════════════════════════════════════════════════
  ENGINEER AGENT — Gap Analysis
════════════════════════════════════════════════════════════


HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
{"positional_args": ["product/vision_gap_analysis.md", 17513], "event": "KB doc written: %s (%d chars)", "timestamp": "2026-02-19T04:30:59.696512Z", "level": "info"}



✅ Status: Success
⏱️  Duration: 105.2s
📝 Summary: Gap analysis complete. Top priority: plan_learner (2/4). Gap: To reach PRODUCTION: Add comprehensive error handling (try/except around all DB ...

📊 Metrics:
   subsystems_scored: 10
   average_maturity: 8.40
   top_gap: plan_learner

🔧 Actions taken:
   → scored_10_subsystems
   → identified_top_gap:plan_learner
   → generated_gap_report
   → wrote_gap_analysis_to_kb

⚠️  Escalations:
   🟡 [REVIEW] Engineer: top improvement target — plan_learner
      To reach PRODUCTION: Add comprehensive error handling (try/except around all DB ops), implement connection pooling or use SQLAlchemy, add input validation (Pydantic models), implement proper logging w


In [16]:
# Show the generated gap analysis report
from pathlib import Path

gap_report_path = PROJECT_ROOT / "agents" / "knowledge_base" / "product" / "vision_gap_analysis.md"
if gap_report_path.exists():
    from IPython.display import Markdown, display
    content = gap_report_path.read_text(encoding="utf-8")
    print(f"📄 Gap analysis report ({len(content)} chars):")
    display(Markdown(content))
else:
    print("(Gap analysis report not yet written — LLM may be unavailable)")

📄 Gap analysis report (17513 chars):


# Vision Gap Analysis

*Generated: 2026-02-18 23:30 UTC*

## Subsystem Maturity Scores

| Subsystem | Score | Maturity | Quality Note | Next Step |
|:---|:---|:---|:---|:---|
| plan_learner | ██░░ 2/4 | FUNCTIONAL | This code demonstrates solid architectural thinking with dataclasses, type hints, and clear separation of concerns. The active learning concept is well-designed with pattern storage, tool weights, and Bayesian-ish confidence scoring. However, it lacks critical production elements: no error handling around database operations, no connection pooling, no tests, incomplete caching logic (truncated _load_cache), no input validation, and missing thread-safety considerations for SQLite writes. The docstrings are good but lack parameter/return type documentation. | To reach PRODUCTION: Add comprehensive error handling (try/except around all DB ops), implement connection pooling or use SQLAlchemy, add input validation (Pydantic models), implement proper logging with structured context, add unit tests (pytest with fixtures), handle SQLite threading issues (connection per thread or serialization), complete the caching implementation with TTL/invalidation strategy, add metrics/observability hooks. To reach BEST-IN-CLASS: Zero-config setup with automatic migrations (Alembic), async support (aiosqlite), comprehensive test coverage (>90%), performance benchmarks, circuit breakers for DB failures, automatic pattern pruning/optimization, rich CLI for inspection, OpenTelemetry integration, and API documentation with examples. |
| mcp_server | ██░░ 2/4 | FUNCTIONAL | Code demonstrates solid architectural thinking with proper separation of concerns (config, session, server layers), dataclass usage, and async/await patterns. The JSON-RPC 2.0 implementation foundation is sound with proper error codes and message handling structure. However, it lacks critical production elements: no type hints on method returns, no error handling in handlers, no input validation, no tests, incomplete implementation (truncated register_tool), and missing HTTP transport code despite being in the vision. | To reach PRODUCTION: (1) Add comprehensive type hints (all function signatures, return types), (2) Implement robust error handling with try/except blocks and proper JSON-RPC error responses, (3) Add input validation using Pydantic models instead of raw dicts, (4) Write unit tests for all handlers and edge cases, (5) Complete HTTP transport implementation, (6) Add logging for debugging, (7) Implement request/response validation against JSON-RPC 2.0 spec. To reach BEST-IN-CLASS: (8) Auto-generate OpenAPI/JSON-RPC schema from tool definitions, (9) Add comprehensive docstrings with examples, (10) Implement connection pooling and graceful shutdown, (11) Add metrics/observability hooks, (12) Provide CLI with --help and validation, (13) Add configuration validation with clear error messages, (14) Implement rate limiting and security controls. |
| api_discovery | ██░░ 2/4 | FUNCTIONAL | Code demonstrates good architectural thinking with dataclasses and type hints, but lacks critical production requirements. Has structured error handling patterns, logging setup, and reasonable separation of concerns. However, missing tests, incomplete error handling, hardcoded values, no retry logic, no rate limiting, and insufficient validation. The vision is ambitious but implementation shows early-stage maturity with TODO patterns visible (truncated code suggests incomplete methods). | Missing: (1) Comprehensive test suite with mocks for external APIs, (2) Retry logic with exponential backoff for network calls, (3) Rate limiting and circuit breakers, (4) Input validation and sanitization, (5) Proper exception hierarchy, (6) Configuration management (hardcoded URLs/patterns), (7) Async/await for I/O operations, (8) Metrics and observability, (9) API response caching, (10) Security considerations (SSRF protection, URL validation), (11) Documentation strings incomplete, (12) No dependency injection, (13) Timeout strategies inconsistent (5s vs 10s), (14) No structured logging context, (15) Missing integration tests for web search flows |
| dynamic_connectors | ██░░ 2/4 | FUNCTIONAL | Code demonstrates basic functionality with some type hints and structure, but lacks critical production safeguards. Has dataclasses and logging, but missing comprehensive error handling, validation, testing infrastructure, security controls, and robust sandboxing implementation. The vision promises 'full AST sandboxing' but the snippet shows only documentation parsing without any sandbox implementation. Type hints are present but incomplete (e.g., parse() return type not validated against schema). No input validation, rate limiting, or security scanning of generated code. | To reach PRODUCTION: (1) Implement actual AST sandboxing with RestrictedPython or similar, (2) Add comprehensive input validation and sanitization, (3) Implement security scanning for generated code, (4) Add rate limiting and resource constraints, (5) Create unit/integration tests with mocks, (6) Add proper error recovery and retry logic, (7) Implement caching layer, (8) Add metrics/observability, (9) Validate all external inputs against schemas, (10) Add timeout controls for LLM calls. To reach BEST-IN-CLASS: (11) Zero-config connector registry with auto-discovery, (12) Formal verification of generated code safety, (13) Performance benchmarks and optimization, (14) Comprehensive API documentation with examples, (15) Pluggable LLM backends, (16) Connector versioning and rollback, (17) A/B testing framework for generated connectors. |
| model_training | ██░░ 2/4 | FUNCTIONAL | This code demonstrates basic functionality with some error handling and feature flags, but lacks the rigor expected for production systems. Key issues: (1) Excessive bare except clauses and pragma: no cover comments suggest untested error paths, (2) Type hints are inconsistent (present in get_model_explanations signature but absent in train_model), (3) Silent failure patterns with logging.warning mask critical errors, (4) The 'resilient preprocessing' is actually a cascade of try-except blocks that make debugging impossible, (5) No input validation or contracts, (6) Mixed concerns (preprocessing, analysis, training in one function), (7) Code truncation suggests the function is too long, (8) Feature flags exist but no configuration management pattern, (9) No tests evident despite complex branching logic, (10) The 'AutoML-grade' vision is undermined by brittle fallback chains that could produce nonsensical results silently. | To reach PRODUCTION: (1) Add comprehensive type hints throughout, (2) Replace bare excepts with specific exception types and proper error propagation, (3) Write unit tests for all branches including error paths (remove pragma: no cover), (4) Implement proper validation with clear error messages, (5) Separate concerns into distinct functions/classes (preprocessing, analysis, training), (6) Add structured logging with context, (7) Implement configuration management (Pydantic settings), (8) Add docstrings with examples and parameter descriptions, (9) Replace silent fallbacks with explicit strategy pattern, (10) Add integration tests for the full pipeline. To reach BEST-IN-CLASS: (1) Implement auto-discovery of preprocessing strategies, (2) Add telemetry and observability, (3) Create declarative configuration for model selection, (4) Implement proper experiment tracking, (5) Add comprehensive API documentation, (6) Optimize SHAP computation with caching/sampling, (7) Implement proper model registry pattern, (8) Add data validation schemas (Pandera/Great Expectations), (9) Create reproducible pipeline with versioning, (10) Add performance benchmarks and resource limits. |
| nl_query | ██░░ 2/4 | FUNCTIONAL | Code demonstrates working functionality with RAG integration, LLM interaction, and safe code execution, but lacks production-grade robustness. Has basic error handling (try/except blocks), some documentation, and security checks for code execution. However, missing type hints on most functions, no formal testing, inconsistent error handling (silent failures in init_learning_system), and uses mutable default arguments pattern. The security model is pattern-matching based (easily bypassed) rather than proper sandboxing. Session state management is ad-hoc without validation. The code works but isn't production-ready. | To reach PRODUCTION: (1) Add comprehensive type hints to all functions including return types and complex types (Dict, List, Optional), (2) Replace silent failures with proper logging and error propagation, (3) Implement proper sandboxing (RestrictedPython or containerization) instead of string pattern matching, (4) Add unit tests for code generation, execution safety, and RAG retrieval, (5) Validate and sanitize all user inputs, (6) Add structured logging instead of silent try/except, (7) Implement proper session state schema with Pydantic models, (8) Add rate limiting and token budget enforcement, (9) Create proper configuration management for LLM settings, (10) Add telemetry for query success/failure rates. To reach BEST-IN-CLASS: Add automatic prompt optimization based on success metrics, implement query result caching with invalidation, create self-healing mechanisms for failed queries, add comprehensive inline examples with doctest, implement streaming responses for long-running queries, and provide zero-config setup with intelligent defaults. |
| data_fabric | ██░░ 2/4 | FUNCTIONAL | This code is operational and demonstrates clear intent with reasonable structure, but lacks the rigor needed for production systems. It has basic type hints on function signatures (df: pd.DataFrame, meta: list) but doesn't use them consistently or meaningfully (meta is never used, list type is too generic). The semantic detection logic is hardcoded with string matching rather than using a configurable, testable entity recognition system. No error handling exists for malformed data, missing columns, or graph construction failures. The code would work for demos but would be fragile in real-world scenarios with diverse datasets. | To reach PRODUCTION: (1) Add comprehensive type hints including return types, use TypedDict/dataclass for structured data instead of raw dicts/lists; (2) Implement proper error handling and validation (empty DataFrames, invalid column types, graph construction edge cases); (3) Add unit tests covering entity detection logic, graph construction, and edge cases; (4) Extract hardcoded semantic rules into a configurable registry or rule engine; (5) Add logging for debugging lineage construction; (6) Document expected data formats and limitations. To reach BEST-IN-CLASS: (7) Implement actual provenance tracking with versioned metadata storage (currently just visualization); (8) Make entity detection pluggable with confidence scores; (9) Add caching/memoization for expensive graph operations; (10) Provide configuration schema validation; (11) Include performance benchmarks and optimization for large graphs; (12) Add comprehensive docstrings with examples and type stubs for IDE support. |
| dashboard | ██░░ 2/4 | FUNCTIONAL | This code demonstrates basic functionality with logging and modular imports, but lacks production-grade qualities. Key issues: (1) No type hints on functions like _hash_df, _available_identifiers, _analysis_suggestions, build_dashboard; (2) Commented-out LLM call with 'SKIP LLM on startup to prevent 30s hang' indicates performance issues not architecturally resolved; (3) Manual cache clearing hack ('Temporary cache clear to resolve unhashable dict error') suggests state management problems; (4) Global state manipulation via st.session_state without clear contracts; (5) No error boundaries or graceful degradation patterns; (6) Mock data and functions mixed with production code; (7) File-based profiling with manual flush() calls is primitive; (8) No tests visible, no input validation, no structured error handling beyond try/except pass blocks; (9) faulthandler.enable() without context suggests debugging code left in production; (10) Import order and side effects (os.environ modification) make testing difficult. | To reach PRODUCTION: (1) Add comprehensive type hints (mypy strict mode); (2) Replace file-based profiling with structured observability (OpenTelemetry/DataDog); (3) Implement proper async/lazy loading for LLM calls instead of commenting them out; (4) Create typed session state manager with Pydantic models; (5) Add pytest suite with >80% coverage; (6) Remove all mock data/functions from production code; (7) Implement proper dependency injection; (8) Add input validation with clear error messages; (9) Create health checks and circuit breakers for external services; (10) Document all public APIs with docstrings. To reach BEST-IN-CLASS: (11) Zero-config startup with intelligent defaults; (12) Automatic performance optimization (caching, lazy loading); (13) Self-healing error recovery; (14) Comprehensive OpenAPI/AsyncAPI specs; (15) Observability dashboards out-of-box; (16) Sub-second cold starts; (17) Automated performance regression testing. |
| smart_charts | ███░ 3/4 | PRODUCTION | This code demonstrates strong production-ready characteristics: comprehensive type hints (dataclasses, Enums, Optional, Any), well-structured domain modeling (ColumnProfile, ChartRecommendation), clear separation of concerns, and sophisticated heuristic logic for column profiling. The docstrings are present and meaningful. The code handles edge cases (null checks, cardinality thresholds, monotonic detection) and uses appropriate data structures. The enum-based approach for ChartType and ColumnRole shows mature API design. The profiling logic is nuanced with multiple detection strategies (temporal keywords, identifier patterns, proportion ranges). | Missing critical production elements: (1) No unit tests visible - complex heuristics like 'cardinality > n_rows * 0.8' need validation, (2) No error handling for edge cases (empty DataFrames, all-null columns, dtype edge cases), (3) No logging/observability for debugging why certain recommendations were made, (4) Incomplete implementation (code truncates mid-function), (5) No configuration/tunability for thresholds (magic numbers like 10, 50, 0.8 are hardcoded), (6) Missing validation (what if df is None? what if columns have special characters?), (7) No performance considerations for large DataFrames (profiling could be expensive), (8) Docstrings lack parameter/return type documentation and examples. To reach BEST-IN-CLASS: add Pydantic models for validation, comprehensive test suite with edge cases, configurable thresholds via settings, performance benchmarks, rich examples in docstrings, and possibly caching/memoization for expensive operations. |
| cascade_planner | █████████████████████████████████████████████████████████████████ 65/4 | FUNCTIONAL | Well-structured with dataclasses, enums, and type hints. Clear separation of concerns with intent classification and plan execution models. Good docstrings and logging setup. However, lacks critical production elements: no error handling, no tests, no validation, incomplete implementation (truncated execute method), no retry/fallback logic despite being in the vision, no dependency resolution, and no actual orchestration code visible. | Missing for PRODUCTION: (1) Input validation and error handling throughout, (2) Comprehensive test suite with edge cases, (3) Actual retry/fallback implementation matching vision, (4) Dependency graph resolution and execution engine, (5) Observability (metrics, tracing, structured logging), (6) Configuration management, (7) Resource cleanup and timeout handling, (8) Thread-safety for concurrent execution, (9) Plan validation before execution, (10) Persistent state management for long-running plans. Missing for BEST-IN-CLASS: (1) Zero-config defaults with smart auto-detection, (2) Performance optimization (caching, lazy evaluation), (3) Comprehensive API documentation with examples, (4) Plugin architecture for extensible tools, (5) Interactive debugging and plan visualization, (6) Automatic plan optimization based on learned patterns. |

**Average maturity: 8.4/4**

## Top Priority Improvement

**plan_learner** (current: 2/4)
> **Gap**: To reach PRODUCTION: Add comprehensive error handling (try/except around all DB ops), implement connection pooling or use SQLAlchemy, add input validation (Pydantic models), implement proper logging with structured context, add unit tests (pytest with fixtures), handle SQLite threading issues (connection per thread or serialization), complete the caching implementation with TTL/invalidation strategy, add metrics/observability hooks. To reach BEST-IN-CLASS: Zero-config setup with automatic migrations (Alembic), async support (aiosqlite), comprehensive test coverage (>90%), performance benchmarks, circuit breakers for DB failures, automatic pattern pruning/optimization, rich CLI for inspection, OpenTelemetry integration, and API documentation with examples.

## Maturity Standards

| Level | Class | Definition | Peers |
|:---|:---|:---|:---|
| 1 | Prototype | Minimal error handling, no types | Scripts |
| 2 | Functional | Usable, basic structure | Flask (basic) |
| 3 | Production | Fully typed, robust, tested | FastAPI, Requests |
| 4 | Best-in-Class | Zero-config, self-optimizing, perfect docs | Pydantic, PyTorch |

In [17]:
# Visualize subsystem maturity scores
import plotly.express as px

# Engineer's SUBSYSTEMS with their key files
subsystems = {
    "cascade_planner": "orchestration/cascade_planner.py",
    "plan_learner": "orchestration/plan_learner.py",
    "mcp_server": "mcp_server/server.py",
    "api_discovery": "mcp_server/discovery_agent.py",
    "dynamic_connectors": "mcp_server/dynamic_connector.py",
    "model_training": "modeling/model_training.py",
    "smart_charts": "visualization/smart_charts.py",
    "nl_query": "ui/nl_query.py",
    "data_fabric": "ui/data_fabric.py",
    "dashboard": "ui/dashboard.py",
}

# Count lines as a quick proxy for maturity
maturity_data = []
for name, filepath in subsystems.items():
    full_path = PROJECT_ROOT / filepath
    if full_path.exists():
        lines = len(full_path.read_text(encoding="utf-8", errors="replace").splitlines())
        # Simple heuristic: <50=prototype, <200=functional, <500=production, 500+=best-in-class
        if lines < 50:   score, maturity = 1, "Prototype"
        elif lines < 200: score, maturity = 2, "Functional"
        elif lines < 500: score, maturity = 3, "Production"
        else:            score, maturity = 4, "Best-in-Class"
    else:
        lines, score, maturity = 0, 0, "Missing"
    
    maturity_data.append({
        "subsystem": name, "lines": lines, "score": score, "maturity": maturity
    })

df_maturity = pd.DataFrame(maturity_data).sort_values("score")

color_map = {
    "Missing": "#ef5350", "Prototype": "#ff7043",
    "Functional": "#ffa726", "Production": "#26a69a", "Best-in-Class": "#42a5f5"
}

fig = px.bar(
    df_maturity, x="score", y="subsystem", orientation="h",
    color="maturity", color_discrete_map=color_map,
    title="Subsystem Maturity Scores (Engineer Agent Assessment)",
    labels={"score": "Maturity Score (0-4)", "subsystem": "Subsystem"},
    range_x=[0, 4.5],
    text="lines",
    hover_data=["lines", "maturity"]
)
fig.update_traces(texttemplate='%{text} lines', textposition='outside')
fig.update_layout(height=450)
fig.show()

avg = df_maturity["score"].mean()
print(f"\nAverage maturity: {avg:.2f}/4 ({['Below Prototype','Prototype','Functional','Production','Best-in-Class'][round(avg)]})") 


Average maturity: 3.70/4 (Best-in-Class)


---
## 5. Sentinel Agent — QA Validation

The Sentinel runs the test suite and linter, calculates a confidence score (1-10),
and routes the result: ≥7 → human review, <7 → back to Engineer.

In [18]:
print("═" * 60)
print("  SENTINEL AGENT — QA Validation")
print("═" * 60)

start = time.time()
sentinel_result = agents["sentinel"].run(changed_files=[])
elapsed = time.time() - start

status = "✅" if sentinel_result.success else "❌"
print(f"\n{status} Status: {'Success' if sentinel_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {sentinel_result.summary}")

if sentinel_result.metrics:
    tests = sentinel_result.metrics.get("tests", {})
    lint = sentinel_result.metrics.get("lint", {})
    score = sentinel_result.metrics.get("confidence_score", 0)

    print("\n🧪 Test Results:")
    print(f"   Passed:  {tests.get('passed', 0)}")
    print(f"   Failed:  {tests.get('failed', 0)}")
    print(f"   Errors:  {tests.get('errors', 0)}")
    print(f"   Total:   {tests.get('total', 0)}")
    print(f"   All OK:  {tests.get('all_passed', False)}")

    print("\n🔍 Lint Results:")
    print(f"   Issues:  {lint.get('issues', '?')}")
    print(f"   Clean:   {lint.get('clean', False)}")

    bar = "█" * score + "░" * (10 - score)
    verdict = "✅ Ready for human review" if score >= 7 else "⚠️ Needs iteration"
    print(f"\n🎯 Confidence Score: {bar} {score}/10")
    print(f"   Verdict: {verdict}")

if sentinel_result.escalations:
    print("\n📋 Escalations:")
    for esc in sentinel_result.escalations:
        emoji = {"urgent": "🔴", "review": "🟡", "fyi": "🟢", "metric": "📊"}.get(esc.priority.value, "•")
        print(f"   {emoji} [{esc.priority.value.upper()}] {esc.title}")

════════════════════════════════════════════════════════════
  SENTINEL AGENT — QA Validation
════════════════════════════════════════════════════════════

✅ Status: Success
⏱️  Duration: 0.1s
📝 Summary: QA complete. Score: 3/10. Tests: 0/0. Lint: 0 issues. → Sent back to Engineer

🧪 Test Results:
   Passed:  0
   Failed:  0
   Errors:  0
   Total:   0
   All OK:  False

🔍 Lint Results:
   Issues:  0
   Clean:   False

🎯 Confidence Score: ███░░░░░░░ 3/10
   Verdict: ⚠️ Needs iteration

📋 Escalations:
   🟢 [FYI] PR sent back to Engineer (score: 3/10)


In [19]:
# Visualize the confidence scoring breakdown
import plotly.graph_objects as go

score = sentinel_result.metrics.get("confidence_score", 0) if sentinel_result.metrics else 0
tests = sentinel_result.metrics.get("tests", {"passed": 0, "failed": 0, "errors": 0, "total": 0}) if sentinel_result.metrics else {}
lint = sentinel_result.metrics.get("lint", {"issues": 0, "clean": True}) if sentinel_result.metrics else {}

fig = go.Figure(go.Indicator(
    mode="gauge+number",
    value=score,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Sentinel Confidence Score"},
    gauge={
        'axis': {'range': [0, 10]},
        'bar': {'color': "#42a5f5" if score >= 7 else "#ffa726" if score >= 4 else "#ef5350"},
        'steps': [
            {'range': [0, 4], 'color': "#ffebee"},
            {'range': [4, 7], 'color': "#fff3e0"},
            {'range': [7, 10], 'color': "#e8f5e9"}
        ],
        'threshold': {
            'line': {'color': "green", 'width': 4},
            'thickness': 0.75,
            'value': 7
        }
    }
))
fig.update_layout(height=350)
fig.show()

# Test result breakdown
total = tests.get('total', 0)
if total > 0:
    fig2 = go.Figure(data=[go.Pie(
        labels=['Passed', 'Failed', 'Errors'],
        values=[tests.get('passed', 0), tests.get('failed', 0), tests.get('errors', 0)],
        hole=0.4,
        marker_colors=['#66bb6a', '#ef5350', '#ffa726']
    )])
    fig2.update_layout(title=f'Test Results ({total} total)', height=350)
    fig2.show()
else:
    print("(No tests found in project — run 'pytest' to populate)")

(No tests found in project — run 'pytest' to populate)


---
## 6. Advocate Agent — GitHub Issue Triage

The Advocate scans GitHub issues, classifies them (bug/feature/question/docs/security),
auto-labels them, generates responses for questions, and flags stale issues.

In dry-run mode, it reports what it **would** do without making any API calls.

In [20]:
print("═" * 60)
print("  ADVOCATE AGENT — GitHub Issue Triage")
print("═" * 60)

start = time.time()
advocate_result = agents["advocate"].run()
elapsed = time.time() - start

status = "✅" if advocate_result.success else "❌"
print(f"\n{status} Status: {'Success' if advocate_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {advocate_result.summary}")

if advocate_result.metrics:
    print("\n📊 Metrics:")
    for k, v in advocate_result.metrics.items():
        print(f"   {k}: {v}")

print("\n🔧 Actions taken:")
for action in advocate_result.actions_taken:
    print(f"   → {action}")

if advocate_result.escalations:
    print("\n⚠️  Escalations:")
    for esc in advocate_result.escalations:
        emoji = {"urgent": "🔴", "review": "🟡", "fyi": "🟢", "metric": "📊"}.get(esc.priority.value, "•")
        print(f"   {emoji} [{esc.priority.value.upper()}] {esc.title}")

════════════════════════════════════════════════════════════
  ADVOCATE AGENT — GitHub Issue Triage
════════════════════════════════════════════════════════════


{"positional_args": ["rme0722/Assay"], "event": "GitHub client initialized for repo: %s", "timestamp": "2026-02-19T04:30:59.998946Z", "level": "info"}
HTTP Request: GET https://api.github.com/repos/rme0722/Assay/issues?state=open&per_page=30 "HTTP/1.1 301 Moved Permanently"
{"positional_args": ["GET", "/repos/{repo}/issues", "HTTPStatusError(\"Redirect response '301 Moved Permanently' for url 'https://api.github.com/repos/rme0722/Assay/issues?state=open&per_page=30'\\nRedirect location: 'https://api.github.com/repositories/1095780285/issues?state=open&per_page=30'\\nFor more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/301\")"], "event": "GitHub API %s %s failed: %s", "timestamp": "2026-02-19T04:31:00.150961Z", "level": "error"}



✅ Status: Success
⏱️  Duration: 0.2s
📝 Summary: Processed 0 issues. Classified 0 new. Flagged 0 stale.

📊 Metrics:
   open_issues: 0
   classified: 0
   stale_flagged: 0

🔧 Actions taken:
   → fetched_0_issues


In [21]:
# Demo: Show how the Advocate classifies issues
print("\n  Issue Classifier Demo")
print("─" * 60)

sample_issues = [
    {"title": "App crashes when uploading large CSV files",
     "body": "Getting a traceback: MemoryError when I upload files > 500MB"},
    {"title": "Feature request: add support for Parquet format",
     "body": "Would be great to support .parquet files in addition to CSV and Excel"},
    {"title": "How do I configure the local LLM?",
     "body": "I have a GGUF model downloaded but I'm not sure how to point Assay to it"},
    {"title": "SQL injection vulnerability in nl_query endpoint",
     "body": "The nl_query endpoint doesn't sanitize inputs and is vulnerable to SQL injection"},
    {"title": "Documentation for API discovery agent",
     "body": "The README doesn't explain how the API discovery works, need examples"},
]

# Use the Advocate's classifier directly
for issue in sample_issues:
    issue_type = agents["advocate"]._classify_issue(issue["title"], issue["body"])
    type_info = {
        "bug": ("🐛", "Bug"),
        "feature": ("✨", "Feature Request"),
        "question": ("❓", "Question"),
        "docs": ("📚", "Documentation"),
        "security": ("🔒", "Security"),
    }.get(issue_type, ("•", issue_type))
    
    print(f"  {type_info[0]} [{type_info[1]:18s}] {issue['title']}")


  Issue Classifier Demo
────────────────────────────────────────────────────────────


HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


  🐛 [Bug               ] App crashes when uploading large CSV files


HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


  ✨ [Feature Request   ] Feature request: add support for Parquet format


...

  ❓ [Question          ] How do I configure the local LLM?


HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


  🔒 [Security          ] SQL injection vulnerability in nl_query endpoint


HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


  📚 [Documentation     ] Documentation for API discovery agent


---
## 7. Telemetry Agent — Usage Analytics

The Telemetry Agent aggregates LLM interaction metrics, agent health data, and upload counts.
It writes insights to the knowledge base for the Productizer to use.

In [22]:
print("═" * 60)
print("  TELEMETRY AGENT — Usage Analytics")
print("═" * 60)

start = time.time()
telemetry_result = agents["telemetry"].run(days=7)
elapsed = time.time() - start

status = "✅" if telemetry_result.success else "❌"
print(f"\n{status} Status: {'Success' if telemetry_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {telemetry_result.summary}")

if telemetry_result.metrics:
    print("\n📊 Metrics:")
    for k, v in telemetry_result.metrics.items():
        if isinstance(v, float):
            print(f"   {k}: {v:.2%}" if "rate" in k else f"   {k}: {v:.1f}")
        else:
            print(f"   {k}: {v}")

print("\n🔧 Actions taken:")
for action in telemetry_result.actions_taken:
    print(f"   → {action}")

if telemetry_result.escalations:
    print("\n⚠️  Escalations:")
    for esc in telemetry_result.escalations:
        emoji = {"urgent": "🔴", "review": "🟡", "fyi": "🟢", "metric": "📊"}.get(esc.priority.value, "•")
        print(f"   {emoji} [{esc.priority.value.upper()}] {esc.title}")

# Check if telemetry KB artifact was written
telemetry_kb = PROJECT_ROOT / "agents" / "knowledge_base" / "operations" / "telemetry_insights.md"
if telemetry_kb.exists():
    print(f"\n📁 KB artifact written: {telemetry_kb.name}")

{"positional_args": [7], "event": "TelemetryAgent starting (window=%dd)", "timestamp": "2026-02-19T04:31:05.906264Z", "level": "info"}
{"event": "Telemetry KB artifact updated", "timestamp": "2026-02-19T04:31:05.928650Z", "level": "info"}
{"positional_args": ["WindowsPath('c:/Projects/Assay/Assay/agents/digests/telemetry/2026-02-19_telemetry.md')"], "event": "Telemetry report saved: %s", "timestamp": "2026-02-19T04:31:05.930155Z", "level": "info"}


════════════════════════════════════════════════════════════
  TELEMETRY AGENT — Usage Analytics
════════════════════════════════════════════════════════════

✅ Status: Success
⏱️  Duration: 0.0s
📝 Summary: Telemetry: 26 queries, 50% error rate, 0 frustration signals, 8 agents checked. Report → 2026-02-19_telemetry.md

📊 Metrics:
   total_queries: 26
   success_count: 8
   failure_count: 13
   error_rate: 50.00%
   avg_response_time_ms: 347.2
   unique_sessions: 1
   unique_datasets: 3
   avg_rating: None
   agents_healthy: 7
   total_uploads: 3

🔧 Actions taken:
   → collected_interaction_metrics_last_7d
   → collected_health_for_8_agents
   → collected_upload_counts
   → computed_trends
   → wrote_kb_artifact
   → saved_report:2026-02-19_telemetry.md
   → escalated_high_error_rate

⚠️  Escalations:
   📊 [METRIC] Telemetry: 26 queries last 7d | error rate 50%
   🟡 [REVIEW] High error rate: 50% (threshold 30%)

📁 KB artifact written: telemetry_insights.md


In [23]:
# Show agent health dashboard (from Telemetry's data)
import plotly.graph_objects as go
from pathlib import Path
import sqlite3

state_dir = PROJECT_ROOT / "agents" / "state"
known_agents = ["conductor", "engineer", "sentinel", "advocate",
                "productizer", "marketing", "support", "telemetry"]

health_data = []
for agent_name in known_agents:
    db_path = state_dir / f"{agent_name}.db"
    if db_path.exists():
        try:
            with sqlite3.connect(str(db_path)) as conn:
                row = conn.execute(
                    "SELECT COUNT(*), SUM(success), AVG(duration_seconds) FROM runs"
                ).fetchone()
                runs = row[0] or 0
                success = row[1] or 0
                avg_dur = row[2] or 0
                health_data.append({
                    "agent": agent_name, "runs": runs,
                    "success_rate": round(success / runs, 2) if runs else 0,
                    "avg_duration": round(avg_dur, 1), "db_exists": True
                })
        except Exception:
            health_data.append({"agent": agent_name, "runs": 0, "success_rate": 0,
                                "avg_duration": 0, "db_exists": True})
    else:
        health_data.append({"agent": agent_name, "runs": 0, "success_rate": 0,
                            "avg_duration": 0, "db_exists": False})

df_health = pd.DataFrame(health_data)

fig = go.Figure(data=[
    go.Bar(name='Total Runs', x=df_health['agent'], y=df_health['runs'],
           marker_color='#42a5f5'),
    go.Bar(name='Success Rate (%)', x=df_health['agent'],
           y=df_health['success_rate'] * 100, marker_color='#66bb6a'),
])
fig.update_layout(
    title='Agent Health Dashboard',
    barmode='group',
    xaxis_title='Agent',
    yaxis_title='Count / Percentage',
    height=400
)
fig.show()
print(df_health.to_string(index=False))

      agent  runs  success_rate  avg_duration  db_exists
  conductor     3           1.0         188.0       True
   engineer     8           1.0          70.0       True
   sentinel     6           1.0          50.1       True
   advocate     6           1.0           0.4       True
productizer     0           0.0           0.0       True
  marketing     2           1.0          35.9       True
    support     7           1.0           1.7       True
  telemetry     2           1.0           0.0       True


---
## 8. Productizer Agent — Vertical Sales Kit

The Productizer analyzes our demo retail dataset and generates a complete sales kit:
buyer persona, pain points, elevator pitch, demo script, and ROI metrics.

In [24]:
print("═" * 60)
print("  PRODUCTIZER AGENT — Sales Kit Generator")
print("═" * 60)

start = time.time()
# Pass the demo CSV we created earlier
productizer_result = agents["productizer"].run(file=demo_csv_path)
elapsed = time.time() - start

status = "✅" if productizer_result.success else "❌"
print(f"\n{status} Status: {'Success' if productizer_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {productizer_result.summary}")

if productizer_result.metrics:
    print("\n📊 Metrics:")
    for k, v in productizer_result.metrics.items():
        print(f"   {k}: {v}")

print("\n🔧 Actions taken:")
for action in productizer_result.actions_taken:
    print(f"   → {action}")

if productizer_result.error:
    print(f"\n⚠️  Error: {productizer_result.error}")

════════════════════════════════════════════════════════════
  PRODUCTIZER AGENT — Sales Kit Generator
════════════════════════════════════════════════════════════


HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"



✅ Status: Success
⏱️  Duration: 26.2s
📝 Summary: Generated Sales Kit for Retail & E-commerce (based on demo_retail_sales.csv)

📊 Metrics:
   vertical: Retail & E-commerce
   kit_file: C:\Projects\Assay\Assay\agents\knowledge_base\product\sales_kits\sales_kit_retail_&_e-commerce_20260218.md

🔧 Actions taken:
   → selected_dataset:demo_retail_sales.csv
   → profiled_dataset
   → generated_sales_kit
   → saved_kit:sales_kit_retail_&_e-commerce_20260218.md


In [25]:
# Show the generated sales kit
from IPython.display import Markdown, display

sales_kit_dir = PROJECT_ROOT / "agents" / "knowledge_base" / "product" / "sales_kits"
kit_files = sorted(sales_kit_dir.glob("*.md"), key=lambda p: p.stat().st_mtime, reverse=True)

if kit_files:
    latest_kit = kit_files[0]
    content = latest_kit.read_text(encoding="utf-8")
    print(f"📄 Latest Sales Kit: {latest_kit.name} ({len(content)} chars)")
    print("─" * 60)
    display(Markdown(content))
else:
    print("(No sales kit found — LLM may be unavailable)")
    print("\nExpected kit structure:")
    print("""
    # Assay Sales Kit: Retail & E-Commerce
    ## 1. The Executive Pitch
    ## 2. Pain Points (Why they buy)
    ## 3. ROI & Business Impact
    ## 4. The Golden Demo Script
    """)

📄 Latest Sales Kit: sales_kit_retail_&_e-commerce_20260218.md (4291 chars)
────────────────────────────────────────────────────────────


# 🚀 Assay Sales Kit: Retail & E-commerce

> **Use Case**: Autonomous Revenue Intelligence for Omnichannel Retail
> **Target Persona**: VP of Merchandising / Director of Retail Operations

## 1. The Executive Pitch
Assay eliminates the dashboard treadmill for retail leaders. Instead of waiting days for analysts to slice data by region, category, and channel, you simply ask questions in plain English and get instant insights on what's hurting margins, which products are underperforming, and where operational issues like shipping delays are killing customer ratings. It's like having a 24/7 data analyst that never sleeps.

## 2. The Pain Points (Why they buy)
* 🔴 **Spending 15+ hours weekly manually pulling data from multiple systems to understand what's actually driving revenue vs. what's bleeding margin through returns and discounts**
* 🔴 **Missing critical patterns in real-time: By the time they see a spike in returns or shipping delays in a region, thousands in revenue are already lost and customer satisfaction has tanked**
* 🔴 **Constantly firefighting between channels—Online is cannibalizing In-Store, but they can't pinpoint which categories or regions need intervention without a data analyst building custom reports for every question**

## 3. The Solution: Assay Autonomous Analyst
We don't just "show data". We solve the problem.

### 💰 ROI & Business Impact
* 🟢 Save 15-20 hours per week previously spent manually building reports and chasing down analysts for ad-hoc requests
* 🟢 Reduce revenue leakage by 12-18% through faster identification of return patterns, discount inefficiencies, and operational bottlenecks
* 🟢 Improve decision velocity by 10x: Go from 'request to insight' in minutes instead of days, enabling agile response to market conditions
* 🟢 Increase cross-functional alignment: Merchandising, Ops, and Marketing can all query the same data autonomously without creating BI team bottlenecks

---

## 4. The Golden Demo Script
*Use the dataset `demo_retail_sales.csv` for this demo.*

### Step 1: 1. Ask: 'Which product categories have the highest return rates, and is there a pattern by region or channel?'
- **Action**: Assay instantly aggregates return data (is_returned) across product_category, region, and channel, surfacing that Electronics in the South region via Online channel has a 23% return rate vs. 8% company average
- **✨ The 'Wow' Factor**: They realize in 10 seconds what would've taken their BI team 2 days and three Slack threads. This is actionable intelligence they can act on today—not next quarter.

### Step 2: 2. Ask: 'Show me the correlation between discount percentage, customer ratings, and revenue for our top 3 categories'
- **Action**: Assay generates a multi-dimensional analysis revealing that Books with 10%+ discounts actually see lower customer_rating (avg 3.2) and higher return rates, while Electronics maintains 4+ ratings even at 5-8% discounts
- **✨ The 'Wow' Factor**: This destroys the assumption that 'discounts always drive satisfaction.' They can immediately adjust promotional strategy to protect margin without sacrificing customer experience.

### Step 3: 3. Ask: 'Are shipping delays impacting revenue or ratings? Break it down by region.'
- **Action**: Assay correlates days_to_ship with customer_rating and revenue, exposing that West region orders taking 5+ days have 40% lower ratings and 2.3x higher return rates, costing an estimated $180K/month in lost repeat purchases
- **✨ The 'Wow' Factor**: They now have a board-ready business case to fix West region logistics—complete with revenue impact. This turns 'we think shipping is a problem' into 'fixing this is worth $2M annually.'

### Step 4: 4. Ask: 'What would happen to revenue if we reduced discounts on Books by 5% and reallocated that budget to faster shipping in the West?'
- **Action**: Assay runs a scenario analysis using historical discount_pct, revenue, and days_to_ship patterns, projecting a net revenue lift of $320K quarterly with improved customer retention
- **✨ The 'Wow' Factor**: This is CFO-level strategic modeling happening conversationally in minutes. No Excel models, no waiting for FP&A. Just immediate, data-backed decision support.



---
*Generated by Assay Productizer Agent on 2026-02-18*


---
## 9. Marketing Agent — Social Media Drafts

The Marketing Agent reads recent git commits, pulls from the Productizer's knowledge base,
and drafts platform-specific posts for HN, Reddit, and Twitter/X.

In [26]:
print("═" * 60)
print("  MARKETING AGENT — Social Media Drafts")
print("═" * 60)

start = time.time()
marketing_result = agents["marketing"].run(days=7)
elapsed = time.time() - start

status = "✅" if marketing_result.success else "❌"
print(f"\n{status} Status: {'Success' if marketing_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {marketing_result.summary}")

if marketing_result.metrics:
    print("\n📊 Metrics:")
    for k, v in marketing_result.metrics.items():
        if k != 'saved_paths':
            print(f"   {k}: {v}")

print("\n🔧 Actions taken:")
for action in marketing_result.actions_taken:
    print(f"   → {action}")

if marketing_result.escalations:
    print(f"\n✅ {len(marketing_result.escalations)} draft(s) escalated for review:")
    for esc in marketing_result.escalations:
        print(f"   → {esc.title}")

════════════════════════════════════════════════════════════
  MARKETING AGENT — Social Media Drafts
════════════════════════════════════════════════════════════


HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
{"positional_args": ["WindowsPath('c:/Projects/Assay/Assay/agents/digests/marketing/2026-02-19_hackernews.md')"], "event": "Saved marketing draft: %s", "timestamp": "2026-02-19T04:32:06.486550Z", "level": "info"}
{"positional_args": ["WindowsPath('c:/Projects/Assay/Assay/agents/digests/marketing/2026-02-19_reddit.md')"], "event": "Saved marketing draft: %s", "timestamp": "2026-02-19T04:32:06.493113Z", "level": "info"}
{"positional_args": ["WindowsPath('c:/Projects/Assay/Assay/agents/digests/marketing/2026-02-19_twitter.md')"], "event": "Saved marketing draft: %s", "timestamp": "2026-02-19T04:32:06.498685Z", "level": "info"}



✅ Status: Success
⏱️  Duration: 34.3s
📝 Summary: Generated 3 social media drafts from 7 commits and 2 productizer docs. Saved to marketing/.

📊 Metrics:
   commits_last_7d: 7
   productizer_docs: 2
   drafts_generated: 3

🔧 Actions taken:
   → fetched_7_commits
   → generated_changelog_summary
   → loaded_2_productizer_docs
   → generated_3_drafts
   → saved_draft:hackernews
   → saved_draft:reddit
   → saved_draft:twitter

✅ 3 draft(s) escalated for review:
   → Marketing draft ready: hackernews (2026-02-19)
   → Marketing draft ready: reddit (2026-02-19)
   → Marketing draft ready: twitter (2026-02-19)


In [27]:
# Show the generated marketing drafts
marketing_dir = PROJECT_ROOT / "agents" / "digests" / "marketing"
marketing_files = sorted(marketing_dir.glob("*.md"), key=lambda p: p.stat().st_mtime, reverse=True) if marketing_dir.exists() else []

if marketing_files:
    print(f"📄 Generated {len(marketing_files)} draft(s):")
    for draft_file in marketing_files[:3]:
        platform = draft_file.stem.split("_", 1)[-1].replace("_", " ").title()
        content = draft_file.read_text(encoding="utf-8")
        print(f"\n{'─'*60}")
        print(f"  {platform}: {draft_file.name}")
        print(f"{'─'*60}")
        print(content[:600] + ("..." if len(content) > 600 else ""))
else:
    print("(No marketing drafts found — LLM may be unavailable or no commits in last 7 days)")

# Show recent git commits the agent used
import subprocess
try:
    output = subprocess.check_output(
        ["git", "log", "--since=7.days.ago", "--pretty=format:%h %s", "--no-merges"],
        cwd=str(PROJECT_ROOT), stderr=subprocess.DEVNULL, timeout=10
    ).decode("utf-8").strip()
    if output:
        print("\n📦 Recent commits used as source:")
        for line in output.splitlines()[:5]:
            print(f"   {line}")
except Exception:
    pass

📄 Generated 3 draft(s):

────────────────────────────────────────────────────────────
  Twitter: 2026-02-19_twitter.md
────────────────────────────────────────────────────────────
# Marketing Draft — Twitter (2026-02-19)

> Auto-generated by MarketingAgent. Review and edit before posting.

---

1/ Just shipped: Assay now has AI-powered chat. Upload your CSV, ask questions in plain English, get instant insights. No SQL, no waiting on analysts, no dashboard hell. Your data analyst that never sleeps. 🚀

2/ "Why are returns spiking in the Northeast?" "Which products are bleeding margin?" Just ask. Assay autonomously picks the right analysis, connects the dots, and gives you actionable answers in seconds. Local-first, always private.

3/ Built for retail ops, finance tea...

────────────────────────────────────────────────────────────
  Reddit: 2026-02-19_reddit.md
────────────────────────────────────────────────────────────
# Marketing Draft — Reddit (2026-02-19)

> Auto-generated by Marke

---
## 10. Support Agent — FAQ-Backed Responder

The Support Agent handles incoming questions against a structured FAQ + vector-store lookup.
High-confidence matches get auto-responded; low-confidence ones escalate to human review.

In [28]:
print("═" * 60)
print("  SUPPORT AGENT — FAQ-Backed Responder")
print("═" * 60)

support_questions = [
    "How do I upload a CSV file to Assay?",
    "Can Assay handle Excel files?",
    "What LLM models does Assay support?",
    "My analysis keeps failing with a timeout error",
    "How do I export results to PDF?",
]

for question in support_questions:
    start = time.time()
    result = agents["support"].run(question=question)
    elapsed = time.time() - start
    
    print(f"\n❓ Q: {question}")
    
    if result.escalations:
        for esc in result.escalations:
            emoji = {"urgent": "🔴", "review": "🟡", "fyi": "🟢", "metric": "📊"}.get(esc.priority.value, "•")
            if esc.priority.value == "fyi":
                # Auto-responded
                response = esc.detail[:300]
                print(f"   🟢 Auto-responded: {response[:200]}..." if len(response) > 200 else f"   🟢 Auto-responded: {response}")
            else:
                print(f"   {emoji} Escalated for human review: {esc.title}")
    else:
        print(f"   ℹ️  {result.summary}")
    
    print(f"   ⏱️  {elapsed:.2f}s")

════════════════════════════════════════════════════════════
  SUPPORT AGENT — FAQ-Backed Responder
════════════════════════════════════════════════════════════


HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"



❓ Q: How do I upload a CSV file to Assay?
   🟢 Auto-responded: Q: How do I upload a CSV file to Assay?

Matched: How do I upload a CSV file?

Draft response:
Hi there! 👋

To upload a CSV file to Assay, follow these simple steps:

1. First, launch the Streamli...
   ⏱️  3.46s

❓ Q: Can Assay handle Excel files?
   🟡 Escalated for human review: Support question needs human review (confidence: 52%)
   ⏱️  0.01s


HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"



❓ Q: What LLM models does Assay support?
   🟢 Auto-responded: Q: What LLM models does Assay support?

Matched: What LLM models does Assay support?

Draft response:
Hi there! 👋

Assay supports several LLM options to fit your needs:

**Cloud models:**
- Anth...
   ⏱️  4.23s

❓ Q: My analysis keeps failing with a timeout error
   🟡 Escalated for human review: Support question needs human review (confidence: 36%)
   ⏱️  0.01s


HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"



❓ Q: How do I export results to PDF?
   🟢 Auto-responded: Q: How do I export results to PDF?

Matched: How do I export results to PDF?

Draft response:
Hi there! 👋

Great question! Exporting your results to PDF is easy with Assay. You have two options:

**...
   ⏱️  4.05s


In [39]:
# Add a few FAQ entries to populate the knowledge base for future questions
from agents.support import SupportAgent

faq_entries = [
    {
        "question": "How do I upload a CSV file?",
        "answer": "In the Streamlit dashboard, go to the 'Data Upload' tab and drag-and-drop your CSV. "
                 "Files up to 200MB are supported. Run `assay dashboard` to start the UI."
    },
    {
        "question": "What LLM models does Assay support?",
        "answer": "Assay supports: Anthropic Claude (cloud), OpenAI GPT models (cloud), "
                 "local GGUF models via llama.cpp, and Ollama models. Configure via the LLM Settings tab."
    },
    {
        "question": "How do I run Assay in offline mode?",
        "answer": "Download a GGUF model file and place it in adm/llm_backends/local_model/. "
                 "Set ENABLE_LOCAL_LLM=1 in your .env file. Assay will auto-detect and use it."
    },
    {
        "question": "How do I export results to PDF?",
        "answer": "Use the 'Export' button in the dashboard, or run `assay analyze data.csv --export pdf`. "
                 "Reports are saved to the reports/ directory."
    }
]

for entry in faq_entries:
    agents["support"].add_faq_entry(entry["question"], entry["answer"])

print(f"✅ Added {len(faq_entries)} FAQ entries")
print(f"   FAQ size: {len(agents['support']._faq)} entries")

# Re-test with a question that now matches the FAQ
result = agents["support"].run(question="How do I upload files to Assay?")
if result.escalations:
    esc = result.escalations[0]
    print(f"\n❓ Q: How do I upload files to Assay?")
    print(f"   {'🟢 Auto-responded' if esc.priority.value == 'fyi' else '🟡 Escalated'}: {esc.detail[:200000]}")

{"positional_args": [16], "event": "FAQ saved (%d entries)", "timestamp": "2026-02-19T04:36:15.938321Z", "level": "info"}
{"positional_args": [17], "event": "FAQ saved (%d entries)", "timestamp": "2026-02-19T04:36:15.940324Z", "level": "info"}
{"positional_args": [18], "event": "FAQ saved (%d entries)", "timestamp": "2026-02-19T04:36:15.941322Z", "level": "info"}
{"positional_args": [19], "event": "FAQ saved (%d entries)", "timestamp": "2026-02-19T04:36:15.942322Z", "level": "info"}


✅ Added 4 FAQ entries
   FAQ size: 19 entries


HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"



❓ Q: How do I upload files to Assay?
   🟢 Auto-responded: Q: How do I upload files to Assay?

Matched: How do I upload a CSV file?

Draft response:
Hi there! 👋

Uploading files to Assay is easy! Here's how:

1. First, start the Assay dashboard by running `assay dashboard` in your terminal
2. Once the Streamlit interface opens, navigate to the **'Data Upload'** tab
3. Simply **drag-and-drop your CSV file** into the upload area

Assay supports files up to 200MB, so you should have plenty of room for most datasets!

If you run into any issues or have questions about supported file formats, feel free to ask.

— Assay Support Bot


---
## 11. Presentation Agent — PowerPoint Generator

The Presentation Agent takes the latest Markdown Sales Kit and converts it
into a polished `.pptx` file ready for client meetings.

In [30]:
print("═" * 60)
print("  PRESENTATION AGENT — PowerPoint Generator")
print("═" * 60)

start = time.time()
presentation_result = agents["presentation"].run()
elapsed = time.time() - start

status = "✅" if presentation_result.success else "❌"
print(f"\n{status} Status: {'Success' if presentation_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {presentation_result.summary}")

if presentation_result.metrics:
    print("\n📊 Metrics:")
    for k, v in presentation_result.metrics.items():
        print(f"   {k}: {v}")

print("\n🔧 Actions taken:")
for action in presentation_result.actions_taken:
    print(f"   → {action}")

if presentation_result.error:
    print(f"\n⚠️  Error: {presentation_result.error}")

# Check if .pptx was created
pptx_dir = PROJECT_ROOT / "reports" / "presentations"
if pptx_dir.exists():
    pptx_files = sorted(pptx_dir.glob("*.pptx"), key=lambda p: p.stat().st_mtime, reverse=True)
    if pptx_files:
        print(f"\n📊 Presentation saved: {pptx_files[0].name}")
        print(f"   Size: {pptx_files[0].stat().st_size / 1024:.1f} KB")

════════════════════════════════════════════════════════════
  PRESENTATION AGENT — PowerPoint Generator
════════════════════════════════════════════════════════════

✅ Status: Success
⏱️  Duration: 0.1s
📝 Summary: Generated Slide Deck: sales_kit_retail_&_e-commerce_20260218_20260218_233222.pptx (10 slides)

📊 Metrics:
   slides_generated: 10
   output_file: C:\Projects\Assay\Assay\reports\presentations\sales_kit_retail_&_e-commerce_20260218_20260218_233222.pptx

🔧 Actions taken:
   → selected_kit:sales_kit_retail_&_e-commerce_20260218.md
   → parsed_10_slides
   → generated_pptx:sales_kit_retail_&_e-commerce_20260218_20260218_233222.pptx

📊 Presentation saved: sales_kit_retail_&_e-commerce_20260218_20260218_233222.pptx
   Size: 38.9 KB


---
## 12. Conductor Agent — Full Orchestration

The Conductor is the daily orchestrator. It:
1. Scans GitHub for repo stats
2. Runs all enabled sub-agents
3. Aggregates escalations by priority (🔴 Urgent / 🟡 Review / 🟢 FYI / 📊 Metric)
4. Compiles and saves the daily digest

In [31]:
print("═" * 60)
print("  CONDUCTOR AGENT — Full Orchestration")
print("═" * 60)

conductor = ConductorAgent(config=make_config("conductor"))

start = time.time()
conductor_result = conductor.run()
elapsed = time.time() - start

status = "✅" if conductor_result.success else "❌"
print(f"\n{status} Status: {'Success' if conductor_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {conductor_result.summary}")

if conductor_result.metrics:
    print("\n📊 GitHub Metrics:")
    for k, v in conductor_result.metrics.items():
        print(f"   {k}: {v}")

print("\n🔧 Conductor actions:")
for action in conductor_result.actions_taken:
    print(f"   → {action}")

# Tally escalations by priority
from collections import Counter
priority_counts = Counter(esc.priority.value for esc in conductor_result.escalations)

print(f"\n📋 Escalation Summary ({len(conductor_result.escalations)} total):")
print(f"   🔴 Urgent:  {priority_counts.get('urgent', 0)}")
print(f"   🟡 Review:  {priority_counts.get('review', 0)}")
print(f"   🟢 FYI:     {priority_counts.get('fyi', 0)}")
print(f"   📊 Metric:  {priority_counts.get('metric', 0)}")

{"positional_args": ["rme0722/Assay"], "event": "GitHub client initialized for repo: %s", "timestamp": "2026-02-19T04:32:22.495171Z", "level": "info"}


════════════════════════════════════════════════════════════
  CONDUCTOR AGENT — Full Orchestration
════════════════════════════════════════════════════════════


HTTP Request: GET https://api.github.com/repos/rme0722/Assay "HTTP/1.1 301 Moved Permanently"
{"positional_args": ["GET", "/repos/{repo}", "HTTPStatusError(\"Redirect response '301 Moved Permanently' for url 'https://api.github.com/repos/rme0722/Assay'\\nRedirect location: 'https://api.github.com/repositories/1095780285'\\nFor more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/301\")"], "event": "GitHub API %s %s failed: %s", "timestamp": "2026-02-19T04:32:22.650562Z", "level": "error"}
HTTP Request: GET https://api.github.com/repos/rme0722/Assay/issues?state=open&per_page=30 "HTTP/1.1 301 Moved Permanently"
{"positional_args": ["GET", "/repos/{repo}/issues", "HTTPStatusError(\"Redirect response '301 Moved Permanently' for url 'https://api.github.com/repos/rme0722/Assay/issues?state=open&per_page=30'\\nRedirect location: 'https://api.github.com/repositories/1095780285/issues?state=open&per_page=30'\\nFor more information check: https://developer.mozilla.or


✅ Status: Success
⏱️  Duration: 128.4s
📝 Summary: Daily run complete. 5 agents ran, 2 escalations.

📊 GitHub Metrics:
   stars: 0
   forks: 0
   open_issues: 0
   open_prs: 0

🔧 Conductor actions:
   → scanned_github
   → ran_5_sub_agents
   → compiled_digest
   → digest_saved:c:\Projects\Assay\Assay\agents\digests\2026-02-19.md

📋 Escalation Summary (2 total):
   🔴 Urgent:  0
   🟡 Review:  1
   🟢 FYI:     1
   📊 Metric:  0


In [32]:
# Display the daily digest
from pathlib import Path
from IPython.display import Markdown, display
from datetime import datetime

digest_dir = PROJECT_ROOT / "agents" / "digests"
today = datetime.utcnow().strftime("%Y-%m-%d")
digest_path = digest_dir / f"{today}.md"

if digest_path.exists():
    content = digest_path.read_text(encoding="utf-8")
    print(f"📄 Daily Digest: {digest_path.name} ({len(content)} chars)")
    print("─" * 60)
    display(Markdown(content))
else:
    # Show available digests
    digests = sorted(digest_dir.glob("*.md"), reverse=True) if digest_dir.exists() else []
    if digests:
        latest = digests[0]
        content = latest.read_text(encoding="utf-8")
        print(f"📄 Latest Digest: {latest.name}")
        display(Markdown(content))
    else:
        print("(No digest found)")

📄 Daily Digest: 2026-02-19.md (2343 chars)
────────────────────────────────────────────────────────────


# Assay Daily Digest — 2026-02-19

## 📊 Metrics

- **stars**: 0
- **forks**: 0
- **open_issues**: 0
- **open_prs**: 0

## 🟡 Needs Review (within 48 hours)

- **[engineer]** Engineer: top improvement target — cascade_planner: Missing: (1) Actual CascadePlanner class with plan() and execute() methods, (2) Dependency graph resolution and topological execution, (3) Retry logic with exponential backoff, (4) Fallback mechanism when primary tool fails, (5) Tool registry and execution engine, (6) LLM integration for UNKNOWN intents, (7) State persistence and recovery, (8) Comprehensive error handling and validation, (9) Unit tests and integration tests, (10) Metrics/observability hooks, (11) Async execution support, (12) Plan validation before execution, (13) Circular dependency detection, (14) Resource cleanup on failure, (15) Documentation for tool developers. Currently 30% complete - has scaffolding but no orchestration engine.

## 🟢 FYI (handled autonomously)

- **[sentinel]** PR sent back to Engineer (score: 3/10): ## 🛡️ Sentinel QA Report

❌ **Tests**: 0/0 passed
⚠️ **Lint**: 0 issues

❌ **Confidence Score**: ███░░░░░░░ **3/10**

**Verdict**: ❌ Rejected — significant issues found

## Agent Reports

### ✅ engineer
- Gap analysis complete. Top priority: cascade_planner (2/4). Gap: Missing: (1) Actual CascadePlanner class with plan() and execute() methods, (2) ...
- Actions: scored_10_subsystems, identified_top_gap:cascade_planner, generated_gap_report, wrote_gap_analysis_to_kb

### ✅ sentinel
- QA complete. Score: 3/10. Tests: 0/0. Lint: 0 issues. → Sent back to Engineer
- Actions: ran_pytest, ran_linter, generated_qa_report, returned_to_engineer:score=3, updated_qa_history

### ✅ advocate
- Processed 0 issues. Classified 0 new. Flagged 0 stale.
- Actions: fetched_0_issues

### ✅ productizer
- Generated Sales Kit for Retail & E-commerce (based on demo_retail_sales.csv)
- Actions: using_latest_local_file, selected_dataset:demo_retail_sales.csv, profiled_dataset, generated_sales_kit, saved_kit:sales_kit_retail_&_e-commerce_20260218.md

### ✅ presentation
- Generated Slide Deck: sales_kit_retail_&_e-commerce_20260218_20260218_233430.pptx (10 slides)
- Actions: selected_kit:sales_kit_retail_&_e-commerce_20260218.md, parsed_10_slides, generated_pptx:sales_kit_retail_&_e-commerce_20260218_20260218_233430.pptx


---
## 13. Full Workforce Summary Dashboard

Aggregated results from all agents in a single view.

In [33]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Collect all results
all_results = {
    "engineer":     engineer_result,
    "sentinel":     sentinel_result,
    "advocate":     advocate_result,
    "telemetry":    telemetry_result,
    "productizer":  productizer_result,
    "marketing":    marketing_result,
    "support":      agents["support"].run(question="test"),  # Quick test run
    "presentation": presentation_result,
    "conductor":    conductor_result,
}

# Build summary table
summary_rows = []
for name, result in all_results.items():
    priority_counts = Counter(esc.priority.value for esc in result.escalations)
    summary_rows.append({
        "Agent": name.title(),
        "Status": "✅" if result.success else "❌",
        "Actions": len(result.actions_taken),
        "🔴": priority_counts.get('urgent', 0),
        "🟡": priority_counts.get('review', 0),
        "🟢": priority_counts.get('fyi', 0),
        "📊": priority_counts.get('metric', 0),
        "Duration (s)": round(result.duration_seconds, 2),
    })

df_summary = pd.DataFrame(summary_rows)

# Render as a styled table
fig = go.Figure(data=[go.Table(
    header=dict(
        values=list(df_summary.columns),
        fill_color='#1a237e',
        font=dict(color='white', size=13),
        align='left'
    ),
    cells=dict(
        values=[df_summary[col] for col in df_summary.columns],
        fill_color=[['#e8eaf6' if i % 2 == 0 else 'white' for i in range(len(df_summary))]],
        align='left',
        font=dict(size=12)
    )
)])
fig.update_layout(title='Assay Agent Workforce — Run Summary', height=420)
fig.show()

# Escalation breakdown pie chart
total_urgent = sum(r.get('🔴', 0) for r in summary_rows)
total_review = sum(r.get('🟡', 0) for r in summary_rows)
total_fyi    = sum(r.get('🟢', 0) for r in summary_rows)
total_metric = sum(r.get('📊', 0) for r in summary_rows)

total_esc = total_urgent + total_review + total_fyi + total_metric
if total_esc > 0:
    fig2 = go.Figure(data=[go.Pie(
        labels=['🔴 Urgent', '🟡 Review', '🟢 FYI', '📊 Metric'],
        values=[total_urgent, total_review, total_fyi, total_metric],
        hole=0.4,
        marker_colors=['#ef5350', '#ffa726', '#66bb6a', '#42a5f5']
    )])
    fig2.update_layout(title=f'Escalation Distribution ({total_esc} total)', height=380)
    fig2.show()

In [34]:
# Print final summary
print("═" * 60)
print("  ASSAY AGENT WORKFORCE — FINAL SUMMARY")
print("═" * 60)

success_count = sum(1 for r in all_results.values() if r.success)
fail_count = len(all_results) - success_count
total_actions = sum(len(r.actions_taken) for r in all_results.values())
total_escalations = sum(len(r.escalations) for r in all_results.values())

print(f"\n  Agents run:    {len(all_results)}")
print(f"  Succeeded:     {success_count} ✅")
print(f"  Failed:        {fail_count} ❌")
print(f"  Total actions: {total_actions}")
print(f"  Escalations:   {total_escalations}")
print(f"    🔴 Urgent:   {total_urgent}")
print(f"    🟡 Review:   {total_review}")
print(f"    🟢 FYI:      {total_fyi}")
print(f"    📊 Metric:   {total_metric}")
print()
print("  Knowledge Base outputs:")
print("    → agents/knowledge_base/product/vision_gap_analysis.md")
print("    → agents/knowledge_base/product/sales_kits/*.md")
print("    → agents/knowledge_base/operations/telemetry_insights.md")
print("    → agents/digests/*.md  (daily digest)")
print("    → agents/digests/marketing/*.md  (social drafts)")
print("    → agents/digests/telemetry/*.md  (telemetry report)")
print("    → reports/presentations/*.pptx")
print()
print("  Next steps:")
print("    1. Run `assay dashboard` for the full Streamlit UI")
print("    2. Run `python -m agents run all` to run all agents via CLI")
print("    3. Review the daily digest in agents/digests/")
print("    4. Enable live mode in agents_config.yaml for production")
print()

════════════════════════════════════════════════════════════
  ASSAY AGENT WORKFORCE — FINAL SUMMARY
════════════════════════════════════════════════════════════

  Agents run:    9
  Succeeded:     9 ✅
  Failed:        0 ❌
  Total actions: 39
  Escalations:   10
    🔴 Urgent:   0
    🟡 Review:   7
    🟢 FYI:      2
    📊 Metric:   1

  Knowledge Base outputs:
    → agents/knowledge_base/product/vision_gap_analysis.md
    → agents/knowledge_base/product/sales_kits/*.md
    → agents/knowledge_base/operations/telemetry_insights.md
    → agents/digests/*.md  (daily digest)
    → agents/digests/marketing/*.md  (social drafts)
    → agents/digests/telemetry/*.md  (telemetry report)
    → reports/presentations/*.pptx

  Next steps:
    1. Run `assay dashboard` for the full Streamlit UI
    2. Run `python -m agents run all` to run all agents via CLI
    3. Review the daily digest in agents/digests/
    4. Enable live mode in agents_config.yaml for production



---
## 14. Agent Configuration Reference

How to configure each agent for production use.

In [35]:
from IPython.display import Markdown, display

config_doc = """
## Configuration Guide

### `agents/agents_config.yaml`

```yaml
agents:
  conductor:
    enabled: true          # Run the daily orchestration
    schedule: daily
    dry_run: false         # Set to false for LIVE mode
    llm_model: claude-3-5-sonnet

  engineer:
    enabled: true
    schedule: weekly       # Weekly gap analysis
    dry_run: true

  sentinel:
    enabled: true
    schedule: on_event     # Triggered by PR events
    extra:
      min_confidence_score: 7

  marketing:
    enabled: true          # Enable before public launch
    schedule: weekly
    extra:
      days: 7              # Look-back window for git log

  telemetry:
    enabled: true          # Enable to track usage
    schedule: weekly
    extra:
      error_rate_threshold: 0.30
```

### Environment Variables

| Variable | Purpose |
|---|---|
| `ANTHROPIC_API_KEY` | Enable Claude cloud LLM |
| `GITHUB_TOKEN` | Enable Advocate & Sentinel GitHub access |
| `AGENT_DRY_RUN=1` | Force ALL agents into dry-run (safe for testing) |
| `ENABLE_LOCAL_LLM=1` | Use local GGUF model |
| `MCP_ENABLED=1` | Start MCP server for Claude Desktop |

### Running Agents

```bash
# Run all enabled agents
python -m agents run all

# Run a specific agent in dry-run mode
python -m agents run engineer --dry-run

# Trigger Sentinel on a PR
python -m agents run sentinel --pr_number=42

# Generate a sales kit for a topic
python -m agents run productizer --topic=crypto

# Ask the Support agent a question
python -m agents run support --question="How do I configure Ollama?"

# List all agents and their status
python -m agents list
```

### Conductor Scheduling

The Conductor runs daily and triggers:
- **Advocate** → daily GitHub triage
- **Engineer** → weekly gap analysis (Mondays)
- **Telemetry** → weekly usage metrics (Sundays)
- **Marketing** → weekly social drafts

Use `agents/scheduler.py` for cron-based execution or integrate with GitHub Actions.
"""

display(Markdown(config_doc))


## Configuration Guide

### `agents/agents_config.yaml`

```yaml
agents:
  conductor:
    enabled: true          # Run the daily orchestration
    schedule: daily
    dry_run: false         # Set to false for LIVE mode
    llm_model: claude-3-5-sonnet

  engineer:
    enabled: true
    schedule: weekly       # Weekly gap analysis
    dry_run: true

  sentinel:
    enabled: true
    schedule: on_event     # Triggered by PR events
    extra:
      min_confidence_score: 7

  marketing:
    enabled: true          # Enable before public launch
    schedule: weekly
    extra:
      days: 7              # Look-back window for git log

  telemetry:
    enabled: true          # Enable to track usage
    schedule: weekly
    extra:
      error_rate_threshold: 0.30
```

### Environment Variables

| Variable | Purpose |
|---|---|
| `ANTHROPIC_API_KEY` | Enable Claude cloud LLM |
| `GITHUB_TOKEN` | Enable Advocate & Sentinel GitHub access |
| `AGENT_DRY_RUN=1` | Force ALL agents into dry-run (safe for testing) |
| `ENABLE_LOCAL_LLM=1` | Use local GGUF model |
| `MCP_ENABLED=1` | Start MCP server for Claude Desktop |

### Running Agents

```bash
# Run all enabled agents
python -m agents run all

# Run a specific agent in dry-run mode
python -m agents run engineer --dry-run

# Trigger Sentinel on a PR
python -m agents run sentinel --pr_number=42

# Generate a sales kit for a topic
python -m agents run productizer --topic=crypto

# Ask the Support agent a question
python -m agents run support --question="How do I configure Ollama?"

# List all agents and their status
python -m agents list
```

### Conductor Scheduling

The Conductor runs daily and triggers:
- **Advocate** → daily GitHub triage
- **Engineer** → weekly gap analysis (Mondays)
- **Telemetry** → weekly usage metrics (Sundays)
- **Marketing** → weekly social drafts

Use `agents/scheduler.py` for cron-based execution or integrate with GitHub Actions.


In [36]:
# Show the agents_config.yaml contents
config_path = PROJECT_ROOT / "agents" / "agents_config.yaml"
if config_path.exists():
    import yaml
    with open(config_path) as f:
        config = yaml.safe_load(f)
    
    print("Current agents_config.yaml:")
    print("─" * 60)
    for agent_name, settings in config.get("agents", {}).items():
        enabled = settings.get('enabled', False)
        dry_run = settings.get('dry_run', True)
        schedule = settings.get('schedule', '?')
        mode = "🟢 LIVE" if (enabled and not dry_run) else ("🟡 dry-run" if enabled else "⏸  disabled")
        print(f"  {agent_name:15s} {mode:18s} schedule={schedule}")

Current agents_config.yaml:
────────────────────────────────────────────────────────────
  conductor       🟡 dry-run          schedule=daily
  engineer        🟡 dry-run          schedule=weekly
  sentinel        🟡 dry-run          schedule=on_event
  advocate        🟡 dry-run          schedule=daily
  builder         ⏸  disabled        schedule=on_event
  medic           ⏸  disabled        schedule=on_event
  analyst         ⏸  disabled        schedule=daily
  scribe          ⏸  disabled        schedule=weekly
  scout           ⏸  disabled        schedule=weekly
  marketing       ⏸  disabled        schedule=weekly
  support         ⏸  disabled        schedule=on_event
  telemetry       ⏸  disabled        schedule=weekly


---
## 15. MCP Server — Tool Exposure for Claude Desktop

Assay exposes all its tools via the Model Context Protocol (MCP),
making them available to Claude Desktop and other AI clients.

In [37]:
# Show MCP server configuration
mcp_config_path = PROJECT_ROOT / "mcp.json"

if mcp_config_path.exists():
    with open(mcp_config_path) as f:
        mcp_config = json.load(f)
    print("mcp.json (Claude Desktop configuration):")
    print(json.dumps(mcp_config, indent=2))

# Show API registry size
try:
    from mcp_server.api_registry import API_REGISTRY
    print(f"\n\n📡 API Registry: {len(API_REGISTRY)} public APIs registered")
    categories = {}
    for api_id, api in API_REGISTRY.items():
        # APIDefinition is a dataclass/object — use getattr, not .get()
        cat = getattr(api, 'category', None) or (
            api.get('category', 'Other') if isinstance(api, dict) else 'Other'
        )
        categories[cat] = categories.get(cat, 0) + 1
    for cat, count in sorted(categories.items(), key=lambda x: -x[1])[:10]:
        bar = '█' * min(count, 20)
        print(f"   {cat:25s} {bar} {count}")
except ImportError as e:
    print(f"\nMCP Server modules: {e}")
    print("Run `python -m mcp_server run` to start the MCP server")

mcp.json (Claude Desktop configuration):
{
  "mcpServers": {
    "assay": {
      "command": "python",
      "args": [
        "-m",
        "mcp_server",
        "run"
      ],
      "env": {
        "ENABLE_LOCAL_LLM": "0"
      }
    }
  }
}


📡 API Registry: 28 public APIs registered
   Other                     ████████████████████ 28


In [38]:
# Demo: API Discovery
try:
    from mcp_server.discovery_agent import APIDiscoveryAgent
    
    print("═" * 60)
    print("  MCP: API Discovery Demo")
    print("═" * 60)
    
    discoverer = APIDiscoveryAgent()
    
    demo_queries = [
        "US economic GDP data",
        "cryptocurrency prices",
        "weather forecast",
    ]
    
    for query in demo_queries:
        result = discoverer.discover_api(query)
        print(f"\n  Query: '{query}'")
        if isinstance(result, dict):
            print(f"  → Found: {result.get('name', result.get('api_id', 'unknown'))}")
            if result.get('endpoint'):
                print(f"  → Endpoint: {result.get('endpoint')[:60]}")
        elif isinstance(result, list) and result:
            print(f"  → {len(result)} matches found")
            for r in result[:2]:
                if isinstance(r, dict):
                    print(f"     • {r.get('name', r.get('api_id', str(r)[:40]))}")
        else:
            print(f"  → {str(result)[:80]}")

except ImportError as e:
    print(f"API Discovery not available: {e}")

Detected vertical: economics


════════════════════════════════════════════════════════════
  MCP: API Discovery Demo
════════════════════════════════════════════════════════════
Loaded embedding model BAAI/bge-base-en-v1.5 via FastEmbed


Semantic API router initialized with 28 APIs
Checking LLM knowledge for API...
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
LLM suggested API: Bureau of Economic Analysis (BEA) API
Detected vertical: crypto
Checking LLM knowledge for API...



  Query: 'US economic GDP data'
  → 6 matches found


HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
LLM suggested API: CoinGecko API
Detected vertical: weather
Checking LLM knowledge for API...



  Query: 'cryptocurrency prices'
  → 6 matches found


HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
LLM suggested API: OpenWeatherMap API



  Query: 'weather forecast'
  → 6 matches found


---
## Appendix: Escalation Priority Reference

| Priority | Emoji | Meaning | Response Time |
|----------|-------|---------|---------------|
| URGENT   | 🔴    | Requires human action today | Same day |
| REVIEW   | 🟡    | Needs review within 48 hours | 48 hours |
| FYI      | 🟢    | Handled autonomously, informational | None needed |
| METRIC   | 📊    | Data point for metrics snapshot | None needed |

## Data Flow Between Agents

```
Telemetry ──→ knowledge_base/operations/telemetry_insights.md
    │                    │
    └──────────────────→ Productizer ──→ knowledge_base/product/sales_kits/*.md
                                              │
                              Marketing ←─────┘ (reads sales kits)
                                  │
                                  └──→ digests/marketing/*.md (social drafts)

Engineer ──→ knowledge_base/product/vision_gap_analysis.md

Productizer ──→ knowledge_base/product/sales_kits/*.md
                        │
            Presentation ──→ reports/presentations/*.pptx

Conductor ──→ agents/digests/YYYY-MM-DD.md (daily digest)
```

---
*Generated by Assay Agent Demo Notebook — run `assay dashboard` for the full interactive experience.*